In [1]:
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

from aesvdd import autoencoder
from aesvdd import model
from aesvdd import datasets

import warnings
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from tensorflow.keras import Model
from tensorflow import keras
import pandas as pd
import numpy as np

import time
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
print(os.getcwd())
warnings.filterwarnings("ignore")


Instructions for updating:
non-resource variables are not supported in the long term
/home/kael99/Documents/Code/Autoencoder_SVDD


In [2]:
# X, y = datasets.get_paysim()
# X.tofile("./data/X.dat")
# y.tofile("./data/y.dat")
# X = np.fromfile("./data/X.dat").reshape(-1,9)
# y = np.fromfile("./data/y.dat", dtype=np.int64)
# X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=60000, test_size=20000)
X_train = np.fromfile("./data/X_train.dat").reshape(-1, 9)
y_train = np.fromfile("./data/y_train.dat", dtype=np.int64)
X_test = np.fromfile("./data/X_test.dat").reshape(-1,9)
y_test = np.fromfile("./data/y_test.dat", dtype=np.int64)


In [3]:
tf.reset_default_graph()
ae_model = autoencoder.VAE(z_dim=5)
# ae_model = autoencoder.Vanilla_AE(H=5)

In [4]:
# pretrain
ae_model.fit(X_train, shuffle=True, epochs=1, batch_size=100, validation_data=(X_test, None), verbose=1)


Train on 60000 samples, validate on 20000 samples
59000/60000 [============================>.] - ETA: 0s - loss: 2.2365WARNING:tensorflow:From /home/kael99/.local/lib/python3.8/site-packages/tensorflow/python/keras/engine/training_v1.py:2048: Model.state_updates (from tensorflow.python.keras.engine.training) is deprecated and will be removed in a future version.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
60000/60000 [==============================] - 1s 21us/sample - loss: 2.2066 - val_loss: 0.3715


In [5]:
inputs = ae_model.input
outputs = ae_model.get_layer(name="code_layer").output
encoder = Model(inputs=inputs, outputs=outputs)

In [6]:
ae_svdd = model.AESVDD(encoder, input_shape=(9,), representation_dim=5, objective="hard", nu=0.05, lr=5e-4)

In [7]:
#train AESVDD
t0 = time.time()
hisory = ae_svdd.fit(X_train, X_test, y_test, epochs=2, verbose=False)
print('Train time:', time.time() - t0)

#test AESVDD
t0 = time.time()
score = ae_svdd.predict(X_test)
auc = roc_auc_score(y_test, -score)
print('Test time:', time.time() - t0)
print('------------- AUROC ----------- : %.4f' % auc)


Instructions for updating:
Simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
Train time: 0.9945864677429199
Test time: 0.08669424057006836
------------- AUROC ----------- : 0.5573


In [9]:
ae_svdd.save_model("1")